In [3]:
import csv
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By
import mysql.connector

In [4]:
#Conexão incial com o banco de dados
conexao =  mysql.connector.connect(host='localhost',user='root',password='root')
print('Conexão com o banco de dados realizada com sucesso!')

#Criação do Schema e das tabelas do banco de dados
cursor = conexao.cursor()
schema = """CREATE SCHEMA IF NOT EXISTS `Raspagem` DEFAULT CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;"""
tables = """CREATE TABLE IF NOT EXISTS `Raspagem`.`Vagas` (
            ID INT NOT NULL AUTO_INCREMENT,
            Vaga VARCHAR(200) NOT NULL,
            Empresa VARCHAR(200) NOT NULL,
            Local VARCHAR(200) NOT NULL,
            Data DATE NOT NULL,
            Descricao MEDIUMTEXT NOT NULL,
            Primary Key (ID)
            ) ENGINE = InnoDB;
        """
cursor.execute(schema)
cursor.execute('use Raspagem')
print('Schema criado com sucesso!')
cursor.execute(tables)
print('Tabelas criadas com sucesso!')
conexao.close()

conexao = mysql.connector.connect(host='localhost', database= 'Raspagem',user='root', password='root')
cursor = conexao.cursor()

Conexão com o banco de dados realizada com sucesso!
Schema criado com sucesso!
Tabelas criadas com sucesso!


In [10]:
#Raspagem Oficial
#Setup para a raspagem
url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&geoId=106057199&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=0'
binary = FirefoxBinary("C:\\Program Files\\Mozilla Firefox\\firefox.exe")
driver = webdriver.Firefox(firefox_binary=binary, executable_path="C:\\geckodriver.exe",)

#Raspagem
vagas = []
#A plataforma permite a visualização de 1000 páginas de vagas, depois da 1000 ele apresenta erro 404
for page in range (18, 1000):
    url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&geoId=106057199&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start='+ str(page)
    driver.get(url)
    time.sleep(4)
    
    #Cada Página apresenta 25 vagas
    for i in range(1, 26):
        #Obtendo os dados da vaga
        Vaga = driver.find_element(By.XPATH, '/html/body/li[' + str(i) + ']/div/div[2]/h3').text
        Empresa = driver.find_element(By.XPATH, '/html/body/li[' + str(i) + ']/div/div[2]/h4/a').text
        Local = driver.find_element(By.XPATH, '/html/body/li['  + str(i) + ']/div/div[2]/div/span').text 
        Data = driver.find_element(By.XPATH, '/html/body/li['   + str(i) + ']/div/div[2]/div/time').get_attribute('datetime')

        #Acessa o Descritivo da Vaga e obtem a descrição dela
        driver.find_element(By.XPATH, '/html/body/li['+ str(i) + ']/div/a/span').click() 
        time.sleep(4)
        driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/button[1]').click()
        Desc = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/div').text
        driver.back()
        time.sleep(4)
        vagas.append({'Vaga' : Vaga, 'Empresa' : Empresa, 'Local' : Local, 'Data' : Data, 'Desc' : Desc})

        #Salva no Banco de Dados
        cursor.execute('INSERT INTO Vagas (Vaga, Empresa, Local, Data, descricao) VALUES (%s, %s, %s, %s, %s)', (Vaga, Empresa, Local, Data, Desc))  
        conexao.commit()

C:\Users\gabriel.ponzio\AppData\Local\Temp\ipykernel_14604\170465390.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary, executable_path="C:\\geckodriver.exe",)
C:\Users\gabriel.ponzio\AppData\Local\Temp\ipykernel_14604\170465390.py:5: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary, executable_path="C:\\geckodriver.exe",)


KeyboardInterrupt: 

In [36]:
#Modelo de Teste
#Setup para a raspagem
url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&geoId=106057199&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start=0'
binary = FirefoxBinary("C:\\Program Files\\Mozilla Firefox\\firefox.exe")
driver = webdriver.Firefox(firefox_binary=binary, executable_path="C:\\geckodriver.exe",)

#Raspagem
vagas = []
#A plataforma permite a visualização de 1000 páginas de vagas, depois da 1000 ele apresenta erro 404
for page in range (0, 2):
    url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=&location=Brasil&geoId=106057199&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0&start='+ str(page)
    driver.get(url)
    time.sleep(2)
    
    #Cada Página apresenta 25 vagas
    
    #Obtendo os dados da vaga
    Vaga = driver.find_element(By.XPATH,  '/html/body/li[' + str(1)+ ']/div/div[2]/h3').text
    Empresa = driver.find_element(By.XPATH, '/html/body/li['+ str(1) + ']/div/div[2]/h4/a').text
    Local = driver.find_element(By.XPATH, '/html/body/li['+ str(1) + ']/div/div[2]/div/span').text
    Data = driver.find_element(By.XPATH, '/html/body/li[' + str(1) + ']/div/div[2]/div/time').get_attribute('datetime')
    #Acessa o Descritivo da Vaga e obtem a descrição dela
    driver.find_element(By.XPATH, '/html/body/li['+ str(1) + ']/div/a').click()
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/button[1]').click()
    Desc = driver.find_element(By.XPATH, '//*[@id="main-content"]/section[1]/div/div/section[1]/div/div/section/div').text
    driver.back()
    time.sleep(1)
    vagas.append({'Vaga' : Vaga, 'Empresa' : Empresa, 'Local' : Local, 'Data' : Data, 'Desc' : Desc})
        #Salva no Banco de Dados
    cursor.execute('INSERT INTO Vagas (Vaga, Empresa, Local, Data, descricao) VALUES (%s, %s, %s, %s, %s)', (Vaga, Empresa, Local, Data, Desc))  
    conexao.commit()

print(vagas[0])
print(vagas[1])

C:\Users\gabriel.ponzio\AppData\Local\Temp\ipykernel_13912\2036957789.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary, executable_path="C:\\geckodriver.exe",)
C:\Users\gabriel.ponzio\AppData\Local\Temp\ipykernel_13912\2036957789.py:5: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary=binary, executable_path="C:\\geckodriver.exe",)


{'Vaga': 'Unisinos POA - Assistente Administrativo', 'Empresa': 'Universidade do Vale do Rio dos Sinos', 'Local': 'Porto Alegre, RS', 'Data': '2023-11-07', 'Desc': 'A equipe de Serviços e Relacionamento da Unisinos busca profissional para atuar no time do Campus POA.\n\nValorizamos a pluralidade, em todos os aspectos, trabalhamos com a convicção de que ela nos possibilita criar vínculos incríveis! Nossas oportunidades independem de gênero, cor/raça, etnia, deficiência, idade, família ou estado civil, identidade ou expressão de gênero, nacionalidade, afiliação política, religião, orientação sexual, dentre outras. Todas as pessoas são bem-vindas no #TimeUnisinos!\n\nPara participar é necessário realizar todas as etapas do cadastro, inclusive os desafios práticos.\n\n\nResponsabilidades e atribuições\n\nAcompanhar resultados da Coordenação, bem como organização de ações visando o atingimento das metas de atendimento;\nApoiar frente aos grupos de trabalho da Coordenação, bem como no suport